<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TP_MachineLearningM1TNSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modèle d'Analyse des Sentiments basé sur un réseau de neurones

L’objectif de ce TP est de développer un modèle prédictif pour la classification des sentiments des critiques de films (données textuelles) à partir d’un apprentissage automatique par réseau de neurones et d'une représentation en sac de mots (bag of words).

## Collection de données de critiques de films 
Le tp utilisera l'ensemble de données issues de 
[Movie Review Polarity Dataset](https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz) que vous devez télécharger.

Ce répertoire contient 1000 revues de films, considérées comme positives (répertoire pos) et 1000 autre considérées négatives (répertoire neg). Ces revues sont au format texte.

90% de ces données seront utilisées en tant que données d'entraînement et 10% seront utilisées en données de test.

Ces fichiers textes sont composés de mots importants (tokens), d'espaces, de signes de ponctuations et de mots de liaisons.

---
Si vous utisez colab, vous devez charger le répertoire txt_sentoken de l'archive à la racine de votre Drive Google.

Puis il vous faut monter le répertoir drive pour qu'il soit accessible par colab.
Exécutez le code suivant,  une clé vous sera demandée. Il vous suffit de suivre le lien, de sélectionner votre profile pour obtenir votre clé que vous copierez dans le champs prévu.

In [0]:
#bloc à exécuter si vous utilisez colab
from google.colab import drive, files

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
## Préparation des données


### Chargement et nettoyage des données

Il convient pour chaque fichier texte de 
  - identifier les termes (**tokens**) entre espaces,
  - supprimer toute ponctuation,
  - supprimer tous les mots qui ne sont pas uniquement composés de caractères alphabétiques,
  - supprimer tous les mots reconnus en tant que mots vides (stop words) (mots de liaison)
  - supprimer tous les mots dont la longueur est <= 1 caractère.

### Les bibliothèques
La préparation des données implique de pouvoir accéder au système de fichier, ainsi qu'à la bibliothèque nltk et aux bibliothèques spécialisées dans le traitement de texte.. : 

In [0]:
##import pour les fichiers et le traitement de données : 
from os import listdir
import nltk
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter
##import pour les réseaux de neurones : 
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import preprocessing
from numpy import array


In [0]:
# le code suivant charge l'ensemble des mots non importants (en anglais)
nltk.download("stopwords")


### Fonctions de lectures et d'"épurage" de fichier texte

In [0]:
def load_doc(filename)->str:
    """retourne le texte inclu dans le fichier filename"""
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
def clean_doc(doc)->list:
    """retourne la liste de mots clés inclus dans le texte doc qui ne font pas parti des stop_words"""
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens
 

### Tests de la récupération de mots clés d'un fichier

In [0]:
rep = '/content/drive/My Drive/txt_sentoken'
# mots clés d'un fichier négatif
filename = rep + '/neg/cv007_4992.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print('les 10 premiers mots cles de ', filename)
print(tokens[:10])
# mots clés d'un fichier positif
filename = rep + '/pos/cv994_12270.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print('les 10 premiers mots cles de ', filename)
print(tokens[:10])

les 10 premiers mots cles de  /Users/emmanueladam/Downloads/review_polarity/txt_sentoken/neg/cv007_4992.txt
['thats', 'exactly', 'long', 'movie', 'felt', 'werent', 'even', 'nine', 'laughs', 'nine']
les 10 premiers mots cles de  /Users/emmanueladam/Downloads/review_polarity/txt_sentoken/pos/cv994_12270.txt
['thriller', 'set', 'modern', 'day', 'seattle', 'marked', 'marky', 'marks', 'migration', 'good']


### Construire le vocabulaire global

In [0]:
def add_doc_to_vocab(filename, vocab):
    """cumule dans la liste vocab les mots du fichier filename 
    (1 seule occurence par mot dans vocab)"""
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

def build_voc(directory, vocab):
    """ajoute au dictionnaire vocab les mots cles des 900 premiers fichiers du repertoire directory"""
    i=0
    # walk through all files in the folder
    for filename in listdir(directory):
        if i>900: break
        # create the absolute filename
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab)
        i = i +1

    
# creer un vocabulaire (liste de mots clés associés à leurs occurences)
vocab = Counter()
# ajouter les mots cles des repertoire pos et neg
build_voc(rep + '/pos', vocab)
build_voc(rep + '/neg', vocab)
# afficher le nb de mots cles trouves
print("nb de mots cles trouves dans les repertoires : ", len(vocab))
# afficher les 20 premiers mots du vocabulaire et leurs nb d'apparitions dans les exemples d'entrainement
print("les 20 premiers mots cles du vocabulaire (et leur nb d'apparition dans les exemples)  : ", end='')
i=0
for (mot,count) in vocab.items(): 
    print(mot,':',count,end=", ")
    i = i+1
    if i>10:break
# afficher les 10 mots cles les plus utilises
print("\nles 10 mots cles les plus utilises : ", vocab.most_common(10))


nb de mots cles trouves dans les repertoires :  44525
les 20 premiers mots cles du vocabulaire (et leur nb d'apparition dans les exemples)  : assume : 45, nothing : 717, phrase : 26, perhaps : 414, one : 4966, used : 338, first : 1624, impressions : 6, rumors : 17, hardly : 114, ever : 665, 
les 10 mots cles les plus utilises :  [('film', 7980), ('one', 4966), ('movie', 4938), ('like', 3182), ('even', 2319), ('good', 2116), ('time', 2052), ('story', 1907), ('films', 1878), ('would', 1856)]


In [0]:
# ne garder que les mots clés apparaissant au moins 3 fois
min_occurrence = 3
tokens = [token for (token,count) in vocab.items() if count >= min_occurrence]
print('en otant les mots utilise moins de ', min_occurrence, ' fois, nb de mots cles = ',len(tokens))

en otant les mots utilise moins de  3  fois, nb de mots cles =  19665


In [0]:
def save_list(lines, filename):
    """sauve les mots de la liste lines dans le fichier filename"""
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'vocab.txt')

---
## Représentation en sac de mots

### Convertir les fichiers en listes de mots-clés appartenant au vocabulaire 


In [0]:
def doc_to_line(filename, vocab)->list:
    """retourne la liste des mots cles du fichier filename appartenant au vocabulaire vocab"""
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [token for token in tokens if token in vocab]
    return ' '.join(tokens)

In [0]:
def process_docs(directory, vocab)->list:
    """retourne deux listes des mots cles du repertoire directory; 
    la 1ere liste represente les 900 premiers fichiers du repertoire, la 2nde represente les 100 derniers"""
    lines_firts = list()
    lines_lasts = list()
    i=1
    # walk through all files in the folder
    for filename in listdir(directory):
        # create the absolute filename
        path = directory + '/' + filename
        # load and clean the doc
        line = doc_to_line(path, vocab)
        if (i<=900): lines_firts.append(line)
        else:lines_lasts.append(line)
        i = i+1
    return (lines_firts,lines_lasts)

In [0]:
# load the vocabulary
vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# load training and testing reviews
(positive_lines_train, positive_lines_test) = process_docs(rep+'/pos', vocab)
(negative_lines_train, negative_lines_test) = process_docs(rep+'/neg', vocab)
# summarize what we have
print("nb exemples d'entrainement positifs : ", len(positive_lines_train))
print("nb exemples d'entrainement negatifs : ", len(negative_lines_train))
print("nb exemples de tests positifs : ", len(positive_lines_test))
print("nb exemples de tests negatifs : ", len(negative_lines_test))

nb exemples d'entrainement positifs :  900
nb exemples d'entrainement negatifs :  900
nb exemples de tests positifs :  100
nb exemples de tests negatifs :  100



### Convertir les listes de mots en vecteur fréquence d'apparition 
#### Créer le sac de mot de l'ensemble d'entrainement

In [0]:
# create the tokenizer
tokenizer = keras.preprocessing.text.Tokenizer()
# load the positive and negative train set into the tokenizer
docs = positive_lines_train + negative_lines_train
tokenizer.fit_on_texts(docs)
# ask to the tokenizer to build the bag of words : a set of (word, frequence of use)*
Xtrain = tokenizer.texts_to_matrix(docs, mode='freq')

In [0]:
print('Xtrain contient ', Xtrain.shape[0], ' exemples de ', Xtrain.shape[1], ' valeurs de fréquence d\'apparition des mots des exemples.')
print('Ainsi, premier exemple d\'entrainement = \n', Xtrain[0])

Xtrain contient  1800  exemples de  19663  valeurs de fréquence d'apparition des mots des exemples.
Ainsi, premier exemple d'entrainement = 
 [0.         0.00477327 0.01431981 ... 0.         0.         0.        ]


On décide que les exemples positifs correspondent à une sortie 0 et que les exemples négatifs correspondent à la sortie 1 :

In [0]:
#ytrain = suites de 0 (classement pour eval positive), suivis d'une suite de 1 (classements pour éval négative)
ytrain = array([0 for _ in range(len(positive_lines_train))] + [1 for _ in range(len(negative_lines_train))])

#### Créer le sac de mot de l'ensemble de test

In [0]:
# tokenize the lines of the test sample
docs = positive_lines_test + negative_lines_test
# ask to the tokenizer to give the bag of words : a set of (word, frequence of use),
# the words are already kown by the tokenizer*
Xtest = tokenizer.texts_to_matrix(docs, mode='freq')
print('Xtest contient ', Xtest.shape[0], ' exemples de ', Xtest.shape[1], ' valeurs de fréquence.')

Xtest contient  200  exemples de  19663  valeurs de fréquence.


In [0]:
#sortie attendues des exemples de test, ytest = suites de 0, suivis d'une suite de 1
ytest = array([0 for _ in range(len(positive_lines_test))] + [1 for _ in range(len(negative_lines_test))])

---
## Modèle de réseau pour l'analyse des sentiments
Le réseau contient en couche d'entrée aurant de neurones que de valeurs retenues

In [0]:
#TODO: donnez le nb de neurones en entrée (= nb de tokens retenus)
n_words = ??

In [0]:
# TODO: définir la structure du réseau
model = Sequential()
# tester différents nb de couches, composés de divers neurones
# tester différentes fonction d'activation (sigmoid, tanh, elu, relu, ...), 
# -> cf. https://keras.io/activations/
model.add(Dense(...

model.add(Dense(...

# compiler le reseau et définir l'optimiseur et la méthode de calcul d'erreur
# tester différents calcul de l'errreur (loss) : mean_squared_error, binary_crossentropy, ..
# -> cf. https://keras.io/losses/
# tester différents optimizer : sgd, adam, adagrad, ...
# -> cf. https://keras.io/optimizers/

model.compile(...)


In [0]:
#TODO tester differents nb de tests (epochs)
model.fit(Xtrain, ytrain, epochs=8, verbose=2)

Epoch 1/8
1800/1800 - 4s - loss: 1.0082 - accuracy: 0.5033
Epoch 2/8
1800/1800 - 4s - loss: 0.5820 - accuracy: 0.8839
Epoch 3/8
1800/1800 - 3s - loss: 0.3947 - accuracy: 0.9261
Epoch 4/8
1800/1800 - 4s - loss: 0.1566 - accuracy: 0.9772
Epoch 5/8
1800/1800 - 4s - loss: 0.0471 - accuracy: 0.9956
Epoch 6/8
1800/1800 - 4s - loss: 0.0134 - accuracy: 1.0000
Epoch 7/8
1800/1800 - 4s - loss: 0.0035 - accuracy: 1.0000
Epoch 8/8
1800/1800 - 4s - loss: 0.0013 - accuracy: 1.0000


In [0]:
# evaluate
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 85.000002


---
## Estimation de nouvelles critiques
Il ne reste plus qu'à utiliser le réseau pour faciliter la classification de futures revues ..  

In [0]:
def predict_sentiment(review, vocab, tokenizer, model):
    """classifie le texte dans la variable 'review' en positif (0) ou negatif (1) """
    # clean
    tokens = clean_doc(review)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    # convert to line
    line = ' '.join(tokens)
    # create the bag of words with the words used in the line
    encoded = tokenizer.texts_to_matrix([line], mode='freq')
    # prediction, output is a vector of (1x1) here (output of the neural network is a vector of 1 element)
    output = model.predict(encoded, verbose=0)
    estimation = round(output[0,0])
    return estimation

In [0]:
# test of a review associated with a note of 10/10
text = 'This movie is the beginning of the culmination of Marvel\'s masterfully woven cinematic universe. Beginning back in 2008 with iron man, we are finally seeing the results of all the movies have been pointing to; and it did not disappoint. Thanos is a complex villain, with deeper and more interesting desires than just "world domination." The dilemmas all the characters face in this movie (both the heroes and the villains) are truly thought provoking and leave you on the edge of your seat. No other set of movies has beeen so involved, so expanded, and encompassed so many story lines/characters and previous movies. The sheer amount of star power alone in this film is insane; and they do a masterful job of weaving all these unique and various characters into a common storyline.'
resultat = predict_sentiment(text, vocab, tokenizer, model)
print('\'',text,'\' ------> est considere comme ', 'positif' if resultat==0 else 'negatif')

# test of a review associated with a note of 1/10
text = 'I was so angry after watching that I had to write something. Boring boring boring, the biggest reason for that is scenario 0/10. It is too long, if I hadn\'t go to cinema and bought ticket there is no way I would have patience to watch it until the end. Everything already seen in previous marvel movies, it seems that every new is even worse with lower quality and I didn\'t even have big expectations before the movie. The main reason I decided to watch it is IMDB rate 9/10 which is total fraud and paid advertisement. There is no guarantee if some character is dead that he will remain dead, everyone could resurrect...Go watch anything else than this. My last marvel movie for sure, so angry that I don\'t watch american movies in cinema any more, all the same, low quality in recent time, just to make many fast, without proper scenario...bull..it'
resultat = predict_sentiment(text, vocab, tokenizer, model)
print('\'',text,'\' ------> est considere comme ', 'positif' if resultat==0 else 'negatif')



' This movie is the beginning of the culmination of Marvel's masterfully woven cinematic universe. Beginning back in 2008 with iron man, we are finally seeing the results of all the movies have been pointing to; and it did not disappoint. Thanos is a complex villain, with deeper and more interesting desires than just "world domination." The dilemmas all the characters face in this movie (both the heroes and the villains) are truly thought provoking and leave you on the edge of your seat. No other set of movies has beeen so involved, so expanded, and encompassed so many story lines/characters and previous movies. The sheer amount of star power alone in this film is insane; and they do a masterful job of weaving all these unique and various characters into a common storyline. ' ------> est considere comme  positif
' I was so angry after watching that I had to write something. Boring boring boring, the biggest reason for that is scenario 0/10. It is too long, if I hadn't go to cinema and 

---
---
## TRAVAIL A RENDRE

* Vous devrez tester différentes architectures de réseau, ainsi que différentes configuration (fonctions d'activation, méthodes de correction d'erreur, méthode de calcul de l'erreur, ...)
* Vous enverrez un fichier contenant pour chaque définition de réseau :
    * l'architecture du réseau (un copier coller du bloc 'TODO: définir la structure du réseau')
    * le déroulé de l'apprentissage (un copier coller du bloc résultat de 'TODO tester differents nb de tests (epochs)'
    * le résultat de 'Test Accuracy'

Au minimum 3 différents réseaux devront être fournis